In [ ]:
"""
Parking Ticket Hotspot Prediction Analysis
Thunder Pandas - DS 4002
Authors: JP Meyer, Nathan Tran, Sanjay Karunamoorthy

This script performs the complete analysis pipeline for predicting parking tickets
in Charlottesville using Poisson regression and Random Forest models.

Requirements:
- pandas
- numpy
- scikit-learn
- matplotlib
- seaborn
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Model imports
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import PoissonRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# Set random seed for reproducibility
np.random.seed(42)

print("="*80)
print("PARKING TICKET HOTSPOT PREDICTION ANALYSIS")
print("Thunder Pandas - DS 4002")
print("="*80)

# =============================================================================
# STEP 1: LOAD AND CLEAN DATA
# =============================================================================
print("\n[STEP 1] Loading and cleaning parking ticket data...")

# Load the data
# NOTE: Update this path to match your data location
df = pd.read_csv('Parking_Tickets.csv')

print(f"Initial dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

# Parse datetime columns
df['DateIssued'] = pd.to_datetime(df['DateIssued'], errors='coerce')
df['TimeIssued'] = pd.to_datetime(df['TimeIssued'], format='%H:%M', errors='coerce').dt.time

# Remove rows with missing critical information
initial_count = len(df)
df = df.dropna(subset=['DateIssued', 'StreetName'])
print(f"Removed {initial_count - len(df)} rows with missing DateIssued or StreetName")

# Standardize street names (remove extra spaces, convert to uppercase)
df['StreetName'] = df['StreetName'].str.strip().str.upper()

# Remove "Void" violations as they are likely administrative errors
df = df[df['ViolationDescription'] != 'Void']
print(f"Removed void tickets. Current shape: {df.shape}")

# Filter to reasonable date range (2010 onwards for better data quality)
df = df[df['DateIssued'] >= '2010-01-01']
print(f"Filtered to 2010 onwards. Current shape: {df.shape}")
df = df[df['DateIssued'] <= pd.Timestamp.now()]
print(f"Filtered to valid dates. Current shape: {df.shape}")

# Remove duplicate tickets
df = df.drop_duplicates(subset=['TicketNumber'])
print(f"After removing duplicates: {df.shape}")

print(f"\nCleaned dataset shape: {df.shape}")
print(f"Date range: {df['DateIssued'].min()} to {df['DateIssued'].max()}")

# =============================================================================
# STEP 2: CREATE TEMPORAL AND STREET-LEVEL FEATURES
# =============================================================================
print("\n[STEP 2] Creating temporal and street-level features...")

# Extract temporal features
df['Year'] = df['DateIssued'].dt.year
df['Month'] = df['DateIssued'].dt.month
df['Day'] = df['DateIssued'].dt.day
df['DayOfWeek'] = df['DateIssued'].dt.dayofweek  # 0=Monday, 6=Sunday
df['DayName'] = df['DateIssued'].dt.day_name()
df['Hour'] = pd.to_datetime(df['TimeIssued'].astype(str), format='%H:%M:%S', errors='coerce').dt.hour
df['IsWeekend'] = df['DayOfWeek'].isin([5, 6]).astype(int)

# Create season feature
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df['Season'] = df['Month'].apply(get_season)

# Create date for aggregation
df['Date'] = df['DateIssued'].dt.date

print(f"Created temporal features: Year, Month, Day, DayOfWeek, Hour, IsWeekend, Season")

# =============================================================================
# STEP 3: AGGREGATE DATA BY STREET AND TIME
# =============================================================================
print("\n[STEP 3] Aggregating ticket counts by street and date...")

# Aggregate by street and date
daily_street = df.groupby(['Date', 'StreetName']).size().reset_index(name='TicketCount')
daily_street['Date'] = pd.to_datetime(daily_street['Date'])

# Add temporal features back
daily_street['Year'] = daily_street['Date'].dt.year
daily_street['Month'] = daily_street['Date'].dt.month
daily_street['DayOfWeek'] = daily_street['Date'].dt.dayofweek
daily_street['IsWeekend'] = daily_street['DayOfWeek'].isin([5, 6]).astype(int)
daily_street['Season'] = daily_street['Month'].apply(get_season)

print(f"Aggregated dataset shape: {daily_street.shape}")
print(f"Date range: {daily_street['Date'].min()} to {daily_street['Date'].max()}")

# Calculate street-level features
street_stats = df.groupby('StreetName').size().reset_index(name='TotalTickets')
street_stats['AvgTicketsPerDay'] = street_stats['TotalTickets'] / \
    (df['Date'].max() - df['Date'].min()).days

# Merge street stats
daily_street = daily_street.merge(street_stats[['StreetName', 'AvgTicketsPerDay']],
                                   on='StreetName', how='left')

# Focus on top streets to reduce noise and improve model performance
top_streets = street_stats.nlargest(50, 'TotalTickets')['StreetName'].tolist()
daily_street = daily_street[daily_street['StreetName'].isin(top_streets)]
print(f"Filtered to top 50 streets. Shape: {daily_street.shape}")

# =============================================================================
# STEP 4: PREPARE FEATURES FOR MODELING
# =============================================================================
print("\n[STEP 4] Preparing features for modeling...")

# Encode categorical variables
le_street = LabelEncoder()
le_season = LabelEncoder()

daily_street['StreetName_Encoded'] = le_street.fit_transform(daily_street['StreetName'])
daily_street['Season_Encoded'] = le_season.fit_transform(daily_street['Season'])

# Sort by date for time series split
daily_street = daily_street.sort_values('Date').reset_index(drop=True)

# Select features for modeling
feature_cols = ['Month', 'DayOfWeek', 'IsWeekend', 'Season_Encoded',
                'StreetName_Encoded', 'AvgTicketsPerDay']
X = daily_street[feature_cols]
y = daily_street['TicketCount']

print(f"Feature matrix shape: {X.shape}")
print(f"Target variable shape: {y.shape}")
print(f"Features used: {feature_cols}")

# =============================================================================
# STEP 5: BUILD NAIVE BASELINE MODELS
# =============================================================================
print("\n[STEP 5] Building naive baseline models...")

def naive_forecast(y_train, y_test):
    """Naive baseline: predict the last observed value"""
    predictions = np.full(len(y_test), y_train.iloc[-1])
    return predictions

def seasonal_naive_forecast(daily_street_train, daily_street_test):
    """Seasonal naive: predict same day of week from previous week"""
    predictions = []

    for idx, row in daily_street_test.iterrows():
        street = row['StreetName']
        dow = row['DayOfWeek']

        # Find the same street and day of week in training data
        historical = daily_street_train[
            (daily_street_train['StreetName'] == street) &
            (daily_street_train['DayOfWeek'] == dow)
        ]

        if len(historical) > 0:
            pred = historical['TicketCount'].mean()
        else:
            pred = daily_street_train['TicketCount'].mean()

        predictions.append(pred)

    return np.array(predictions)

# =============================================================================
# STEP 6: TRAIN AND EVALUATE MODELS WITH TIME SERIES CROSS-VALIDATION
# =============================================================================
print("\n[STEP 6] Training and evaluating models with time series cross-validation...")

# Use TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Storage for results
naive_rmse_scores = []
naive_mape_scores = []
seasonal_rmse_scores = []
seasonal_mape_scores = []
poisson_rmse_scores = []
poisson_mape_scores = []
rf_rmse_scores = []
rf_mape_scores = []

fold = 1
for train_idx, test_idx in tscv.split(X):
    print(f"\n--- Fold {fold} ---")

    # Split data
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    daily_train = daily_street.iloc[train_idx]
    daily_test = daily_street.iloc[test_idx]

    # Naive baseline
    naive_pred = naive_forecast(y_train, y_test)
    naive_rmse = np.sqrt(mean_squared_error(y_test, naive_pred))
    naive_mape = mean_absolute_percentage_error(y_test, naive_pred) * 100
    naive_rmse_scores.append(naive_rmse)
    naive_mape_scores.append(naive_mape)

    # Seasonal naive baseline
    seasonal_pred = seasonal_naive_forecast(daily_train, daily_test)
    seasonal_rmse = np.sqrt(mean_squared_error(y_test, seasonal_pred))
    seasonal_mape = mean_absolute_percentage_error(y_test, seasonal_pred) * 100
    seasonal_rmse_scores.append(seasonal_rmse)
    seasonal_mape_scores.append(seasonal_mape)

    # Poisson Regression
    poisson_model = PoissonRegressor(max_iter=500)
    poisson_model.fit(X_train, y_train)
    poisson_pred = poisson_model.predict(X_test)
    poisson_rmse = np.sqrt(mean_squared_error(y_test, poisson_pred))
    poisson_mape = mean_absolute_percentage_error(y_test, poisson_pred) * 100
    poisson_rmse_scores.append(poisson_rmse)
    poisson_mape_scores.append(poisson_mape)

    # Random Forest Regressor
    rf_model = RandomForestRegressor(n_estimators=100, max_depth=10,
                                     min_samples_split=10, random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    rf_pred = rf_model.predict(X_test)
    rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
    rf_mape = mean_absolute_percentage_error(y_test, rf_pred) * 100
    rf_rmse_scores.append(rf_rmse)
    rf_mape_scores.append(rf_mape)

    print(f"Naive RMSE: {naive_rmse:.3f}, MAPE: {naive_mape:.2f}%")
    print(f"Seasonal Naive RMSE: {seasonal_rmse:.3f}, MAPE: {seasonal_mape:.2f}%")
    print(f"Poisson RMSE: {poisson_rmse:.3f}, MAPE: {poisson_mape:.2f}%")
    print(f"Random Forest RMSE: {rf_rmse:.3f}, MAPE: {rf_mape:.2f}%")

    fold += 1

# =============================================================================
# STEP 7: SUMMARIZE RESULTS
# =============================================================================
print("\n" + "="*80)
print("CROSS-VALIDATION RESULTS SUMMARY")
print("="*80)

results_df = pd.DataFrame({
    'Model': ['Naive Baseline', 'Seasonal Naive', 'Poisson Regression', 'Random Forest'],
    'Avg RMSE': [np.mean(naive_rmse_scores), np.mean(seasonal_rmse_scores),
                 np.mean(poisson_rmse_scores), np.mean(rf_rmse_scores)],
    'Std RMSE': [np.std(naive_rmse_scores), np.std(seasonal_rmse_scores),
                 np.std(poisson_rmse_scores), np.std(rf_rmse_scores)],
    'Avg MAPE (%)': [np.mean(naive_mape_scores), np.mean(seasonal_mape_scores),
                     np.mean(poisson_mape_scores), np.mean(rf_mape_scores)],
    'Std MAPE (%)': [np.std(naive_mape_scores), np.std(seasonal_mape_scores),
                     np.std(poisson_mape_scores), np.std(rf_mape_scores)]
})

print("\n", results_df.to_string(index=False))

# Calculate improvement over naive baseline
baseline_rmse = results_df.loc[0, 'Avg RMSE']
poisson_improvement = ((baseline_rmse - results_df.loc[2, 'Avg RMSE']) / baseline_rmse) * 100
rf_improvement = ((baseline_rmse - results_df.loc[3, 'Avg RMSE']) / baseline_rmse) * 100

print(f"\n--- IMPROVEMENT OVER NAIVE BASELINE ---")
print(f"Poisson Regression: {poisson_improvement:.2f}% improvement in RMSE")
print(f"Random Forest: {rf_improvement:.2f}% improvement in RMSE")

if poisson_improvement >= 15 or rf_improvement >= 15:
    print(f"\n✓ SUCCESS: Goal of 15% improvement achieved!")
else:
    print(f"\n⚠ Goal of 15% improvement not fully achieved. Consider feature engineering.")

# =============================================================================
# STEP 8: TRAIN FINAL MODELS ON FULL DATA
# =============================================================================
print("\n[STEP 8] Training final models on full dataset...")

# Split into train/test (80/20)
split_idx = int(0.8 * len(X))
X_train_final = X.iloc[:split_idx]
X_test_final = X.iloc[split_idx:]
y_train_final = y.iloc[:split_idx]
y_test_final = y.iloc[split_idx:]

# Train final Poisson model
final_poisson = PoissonRegressor(max_iter=500)
final_poisson.fit(X_train_final, y_train_final)
poisson_final_pred = final_poisson.predict(X_test_final)

# Train final Random Forest model
final_rf = RandomForestRegressor(n_estimators=200, max_depth=15,
                                 min_samples_split=10, random_state=42, n_jobs=-1)
final_rf.fit(X_train_final, y_train_final)
rf_final_pred = final_rf.predict(X_test_final)

print("Final models trained successfully.")

# =============================================================================
# STEP 9: GENERATE VISUALIZATIONS
# =============================================================================
print("\n[STEP 9] Generating visualizations...")

# 1. Model Comparison Bar Chart
plt.figure(figsize=(10, 6))
models = results_df['Model']
rmse_values = results_df['Avg RMSE']
colors = ['#d62728', '#ff7f0e', '#2ca02c', '#1f77b4']

plt.bar(models, rmse_values, color=colors, alpha=0.7, edgecolor='black')
plt.axhline(y=baseline_rmse, color='red', linestyle='--', label='Naive Baseline')
plt.xlabel('Model', fontsize=12, fontweight='bold')
plt.ylabel('Average RMSE', fontsize=12, fontweight='bold')
plt.title('Model Performance Comparison (Cross-Validation)', fontsize=14, fontweight='bold')
plt.xticks(rotation=15, ha='right')
plt.legend()
plt.tight_layout()
plt.savefig('model_comparison_rmse.png', dpi=300, bbox_inches='tight')
print("Saved: model_comparison_rmse.png")
plt.close()

# 2. Predictions vs Actual (Random Forest)
plt.figure(figsize=(10, 6))
plt.scatter(y_test_final, rf_final_pred, alpha=0.5, s=20)
plt.plot([y_test_final.min(), y_test_final.max()],
         [y_test_final.min(), y_test_final.max()],
         'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual Ticket Count', fontsize=12, fontweight='bold')
plt.ylabel('Predicted Ticket Count', fontsize=12, fontweight='bold')
plt.title('Random Forest: Predicted vs Actual Ticket Counts', fontsize=14, fontweight='bold')
plt.legend()
plt.tight_layout()
plt.savefig('rf_predictions_vs_actual.png', dpi=300, bbox_inches='tight')
print("Saved: rf_predictions_vs_actual.png")
plt.close()

# 3. Feature Importance (Random Forest)
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': final_rf.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Feature'], feature_importance['Importance'], color='steelblue')
plt.xlabel('Importance Score', fontsize=12, fontweight='bold')
plt.ylabel('Feature', fontsize=12, fontweight='bold')
plt.title('Random Forest Feature Importance', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
print("Saved: feature_importance.png")
plt.close()

# 4. Residual Plot
residuals = y_test_final - rf_final_pred
plt.figure(figsize=(10, 6))
plt.scatter(rf_final_pred, residuals, alpha=0.5, s=20)
plt.axhline(y=0, color='r', linestyle='--', lw=2)
plt.xlabel('Predicted Ticket Count', fontsize=12, fontweight='bold')
plt.ylabel('Residuals', fontsize=12, fontweight='bold')
plt.title('Random Forest Residual Plot', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('residual_plot.png', dpi=300, bbox_inches='tight')
print("Saved: residual_plot.png")
plt.close()

# =============================================================================
# STEP 10: SAVE RESULTS
# =============================================================================
print("\n[STEP 10] Saving results...")

# Save model performance metrics
results_df.to_csv('model_performance_metrics.csv', index=False)
print("Saved: model_performance_metrics.csv")

# Save feature importance
feature_importance.to_csv('feature_importance.csv', index=False)
print("Saved: feature_importance.csv")

# Save final predictions
predictions_df = pd.DataFrame({
    'Actual': y_test_final.values,
    'Poisson_Predicted': poisson_final_pred,
    'RandomForest_Predicted': rf_final_pred,
    'Poisson_Error': y_test_final.values - poisson_final_pred,
    'RF_Error': y_test_final.values - rf_final_pred
})
predictions_df.to_csv('../OUTPUT/final_predictions.csv', index=False)
print("Saved: final_predictions.csv")

print("\n" + "="*80)
print("ANALYSIS COMPLETE")
print("="*80)
print(f"\nAll outputs saved to ../OUTPUT/ directory")
print(f"Review the visualizations and metrics to assess model performance.")
print(f"\nNext steps:")
print(f"1. Review OUTPUT folder for all generated figures and tables")
print(f"2. Update README.md with reproduction instructions")
print(f"3. Prepare presentation materials using these results")
print("="*80)

PARKING TICKET HOTSPOT PREDICTION ANALYSIS
Thunder Pandas - DS 4002

[STEP 1] Loading and cleaning parking ticket data...
Initial dataset shape: (506311, 16)
Columns: ['RecordID', 'TicketNumber', 'DateIssued', 'StreetName', 'TimeIssued', 'StreetNumber', 'LicenseState', 'WaiverRequestDate', 'WaiverGrantedDate', 'AppealDate', 'AppealGrantedDate', 'ViolationDescription', 'AppealStatus', 'Location', 'LicensePlateAnon', 'WaiverStatus']
Removed 5075 rows with missing DateIssued or StreetName
Removed void tickets. Current shape: (493346, 16)
Filtered to 2010 onwards. Current shape: (215843, 16)
After removing duplicates: (215612, 16)

Cleaned dataset shape: (215612, 16)
Date range: 2010-01-01 05:00:00+00:00 to 2208-11-08 05:00:00+00:00

[STEP 2] Creating temporal and street-level features...
Created temporal features: Year, Month, Day, DayOfWeek, Hour, IsWeekend, Season

[STEP 3] Aggregating ticket counts by street and date...
Aggregated dataset shape: (80784, 8)
Date range: 2010-01-01 00:00: